<span style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">An Exception was encountered at '<a href="#papermill-error-cell">In [19]</a>'.</span>

In [1]:
import pycisTopic
import glob
import os
import pybiomart as pbm
import pandas as pd
import pickle
from pycisTopic.qc import *
from IPython.display import Image, display
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

import multiprocess as mp  # for kde multithreading calculation
from multiprocess import Pool

%matplotlib inline
%load_ext lab_black

# Download annotation

In [2]:
!pwd

/lustre1/project/stg_00090/scatac_benchmark/public_4_cistopic_consensus


In [3]:
wdir = "/lustre1/project/stg_00090/scatac_benchmark/public_4_cistopic_consensus"
os.chdir(wdir)

In [4]:
genome = "mm10"

pbm_genome_name_dict = {
    "hg38": "hsapiens_gene_ensembl",
    "hg37": "hsapiens_gene_ensembl",
    "mm10": "mmusculus_gene_ensembl",
    "dm6": "dmelanogaster_gene_ensembl",
}

pbm_host_dict = {
    "hg38": "http://www.ensembl.org",
    "hg37": "http://grch37.ensembl.org/",
    "mm10": "http://nov2020.archive.ensembl.org/",
    "dm6": "http://www.ensembl.org",
}

if os.path.exists(f"annotation.tsv"):
    print(f"Loading cached genome annotation...")
    annotation = pd.read_csv("annotation.tsv", sep="\t", header=0, index_col=0)
else:
    dataset = pbm.Dataset(name=pbm_genome_name_dict[genome], host=pbm_host_dict[genome])

    annotation = dataset.query(
        attributes=[
            "chromosome_name",
            "transcription_start_site",
            "strand",
            "external_gene_name",
            "transcript_biotype",
        ]
    )
    filter = annotation["Chromosome/scaffold name"].str.contains("CHR|GL|JH|MT")
    annotation = annotation[~filter]
    annotation["Chromosome/scaffold name"] = annotation[
        "Chromosome/scaffold name"
    ].str.replace(r"(\b\S)", r"chr\1")
    annotation.columns = ["Chromosome", "Start", "Strand", "Gene", "Transcript_type"]
    annotation = annotation[annotation.Transcript_type == "protein_coding"]
    annotation.to_csv("annotation.tsv", sep="\t")

Loading cached genome annotation...


In [5]:
fragments_list = sorted(
    glob.glob("../1_data_repository/publicdata_full_fragments_vsn/*.tsv.gz")
)
fragments_dict = {}
for fragments_file in fragments_list:
    sample = fragments_file.split("/")[-1].split("_fragments.tsv.gz")[0]
    fragments_dict[sample] = fragments_file
fragments_dict

{'BIO_ddseq_m1c1.FULL.fragments.tsv.gz': '../1_data_repository/publicdata_full_fragments_vsn/BIO_ddseq_m1c1.FULL.fragments.tsv.gz',
 'BIO_ddseq_m1c2.FULL.fragments.tsv.gz': '../1_data_repository/publicdata_full_fragments_vsn/BIO_ddseq_m1c2.FULL.fragments.tsv.gz',
 'BIO_ddseq_m1c3.FULL.fragments.tsv.gz': '../1_data_repository/publicdata_full_fragments_vsn/BIO_ddseq_m1c3.FULL.fragments.tsv.gz',
 'BIO_ddseq_m1c4.FULL.fragments.tsv.gz': '../1_data_repository/publicdata_full_fragments_vsn/BIO_ddseq_m1c4.FULL.fragments.tsv.gz',
 'BIO_ddseq_m1c5.FULL.fragments.tsv.gz': '../1_data_repository/publicdata_full_fragments_vsn/BIO_ddseq_m1c5.FULL.fragments.tsv.gz',
 'BIO_ddseq_m1c6.FULL.fragments.tsv.gz': '../1_data_repository/publicdata_full_fragments_vsn/BIO_ddseq_m1c6.FULL.fragments.tsv.gz',
 'BIO_ddseq_m1c7.FULL.fragments.tsv.gz': '../1_data_repository/publicdata_full_fragments_vsn/BIO_ddseq_m1c7.FULL.fragments.tsv.gz',
 'BIO_ddseq_m1c8.FULL.fragments.tsv.gz': '../1_data_repository/publicdata_fu

The regions we will count in. Here, I count everything in the human ENCODE SCREEN regions. You can also add a sample-specific region set, e.g. sample-specific peaks called on the sample's bams.

Now, make a sub dictionary of all samples within the fragments dict that have not been run yet (good for resuming a stopped cistopic run):

In [6]:
cistopic_qc_out = os.path.join(wdir, "cistopic_qc_out")
if not os.path.exists(cistopic_qc_out):
    os.makedirs(cistopic_qc_out)

In [7]:
region = "../0_resources/regions/V2.mm10-rDHS-Unfiltered.blacklisted.bed"

In [8]:
fragments_sub_dict = {}
regions_sub_dict = {}
for sample in fragments_dict:
    metadata_file = os.path.join(cistopic_qc_out, sample + "__metadata_bc.pkl")
    print(f"Checking if {metadata_file} exist...")
    if os.path.exists(metadata_file):
        print("\tMetadata exists! Skipping...")
    else:
        fragments_sub_dict[sample] = fragments_dict[sample]
        print("\tMetadata does not exist, adding to subdict to generate")

Checking if /lustre1/project/stg_00090/scatac_benchmark/public_4_cistopic_consensus/cistopic_qc_out/BIO_ddseq_m1c1.FULL.fragments.tsv.gz__metadata_bc.pkl exist...
	Metadata does not exist, adding to subdict to generate
Checking if /lustre1/project/stg_00090/scatac_benchmark/public_4_cistopic_consensus/cistopic_qc_out/BIO_ddseq_m1c2.FULL.fragments.tsv.gz__metadata_bc.pkl exist...
	Metadata does not exist, adding to subdict to generate
Checking if /lustre1/project/stg_00090/scatac_benchmark/public_4_cistopic_consensus/cistopic_qc_out/BIO_ddseq_m1c3.FULL.fragments.tsv.gz__metadata_bc.pkl exist...
	Metadata does not exist, adding to subdict to generate
Checking if /lustre1/project/stg_00090/scatac_benchmark/public_4_cistopic_consensus/cistopic_qc_out/BIO_ddseq_m1c4.FULL.fragments.tsv.gz__metadata_bc.pkl exist...
	Metadata does not exist, adding to subdict to generate
Checking if /lustre1/project/stg_00090/scatac_benchmark/public_4_cistopic_consensus/cistopic_qc_out/BIO_ddseq_m1c5.FULL.frag

In [9]:
regions_sub_dict = {x: region for x in sorted(fragments_sub_dict.keys())}

In [10]:
regions_paths_dict = {
    x.split("/")[-1].split(f"__")[0].replace(".FULL", ""): x
    for x in sorted(glob.glob("../public_3_cistopic_qc/final_consensus_peaks/*.bed"))
}
regions_paths_dict

{'BIO_ddseq_m1c1': '../public_3_cistopic_qc/final_consensus_peaks/BIO_ddseq_m1c1.FULL__SCREEN_consensus_peaks.bed',
 'BIO_ddseq_m1c2': '../public_3_cistopic_qc/final_consensus_peaks/BIO_ddseq_m1c2.FULL__SCREEN_consensus_peaks.bed',
 'BIO_ddseq_m1c3': '../public_3_cistopic_qc/final_consensus_peaks/BIO_ddseq_m1c3.FULL__SCREEN_consensus_peaks.bed',
 'BIO_ddseq_m1c4': '../public_3_cistopic_qc/final_consensus_peaks/BIO_ddseq_m1c4.FULL__SCREEN_consensus_peaks.bed',
 'BIO_ddseq_m1c5': '../public_3_cistopic_qc/final_consensus_peaks/BIO_ddseq_m1c5.FULL__SCREEN_consensus_peaks.bed',
 'BIO_ddseq_m1c6': '../public_3_cistopic_qc/final_consensus_peaks/BIO_ddseq_m1c6.FULL__SCREEN_consensus_peaks.bed',
 'BIO_ddseq_m1c7': '../public_3_cistopic_qc/final_consensus_peaks/BIO_ddseq_m1c7.FULL__SCREEN_consensus_peaks.bed',
 'BIO_ddseq_m1c8': '../public_3_cistopic_qc/final_consensus_peaks/BIO_ddseq_m1c8.FULL__SCREEN_consensus_peaks.bed',
 'BIO_ddseq_m2c1': '../public_3_cistopic_qc/final_consensus_peaks/BIO_dd

In [11]:
regions_sub_dict = {x: regions_paths_dict[x.split(".")[0]] for x in fragments_dict}
regions_sub_dict

{'BIO_ddseq_m1c1.FULL.fragments.tsv.gz': '../public_3_cistopic_qc/final_consensus_peaks/BIO_ddseq_m1c1.FULL__SCREEN_consensus_peaks.bed',
 'BIO_ddseq_m1c2.FULL.fragments.tsv.gz': '../public_3_cistopic_qc/final_consensus_peaks/BIO_ddseq_m1c2.FULL__SCREEN_consensus_peaks.bed',
 'BIO_ddseq_m1c3.FULL.fragments.tsv.gz': '../public_3_cistopic_qc/final_consensus_peaks/BIO_ddseq_m1c3.FULL__SCREEN_consensus_peaks.bed',
 'BIO_ddseq_m1c4.FULL.fragments.tsv.gz': '../public_3_cistopic_qc/final_consensus_peaks/BIO_ddseq_m1c4.FULL__SCREEN_consensus_peaks.bed',
 'BIO_ddseq_m1c5.FULL.fragments.tsv.gz': '../public_3_cistopic_qc/final_consensus_peaks/BIO_ddseq_m1c5.FULL__SCREEN_consensus_peaks.bed',
 'BIO_ddseq_m1c6.FULL.fragments.tsv.gz': '../public_3_cistopic_qc/final_consensus_peaks/BIO_ddseq_m1c6.FULL__SCREEN_consensus_peaks.bed',
 'BIO_ddseq_m1c7.FULL.fragments.tsv.gz': '../public_3_cistopic_qc/final_consensus_peaks/BIO_ddseq_m1c7.FULL__SCREEN_consensus_peaks.bed',
 'BIO_ddseq_m1c8.FULL.fragments.ts

In [12]:
ray.shutdown()

In [13]:
n_cores = 5
if fragments_sub_dict != {}:
    samples_sub = list(fragments_sub_dict.keys())
    blocks = [samples_sub[i : i + n_cores] for i in range(0, len(samples_sub), n_cores)]
    for samples_torun_in_block in blocks:
        fragments_sub_dict_block = {
            key: fragments_sub_dict[key] for key in samples_torun_in_block
        }
        regions_sub_dict_block = {
            key: regions_sub_dict[key] for key in samples_torun_in_block
        }

        metadata_bc_dict, profile_data_dict = compute_qc_stats(
            fragments_dict=fragments_sub_dict_block,
            tss_annotation=annotation,
            stats=[
                "barcode_rank_plot",
                "duplicate_rate",
                "insert_size_distribution",
                "profile_tss",
                "frip",
            ],
            label_list=None,
            path_to_regions=regions_sub_dict_block,
            n_cpu=n_cores,
            valid_bc=None,
            n_frag=10,
            n_bc=None,
            tss_flank_window=2000,
            tss_window=50,
            tss_minimum_signal_window=100,
            tss_rolling_window=10,
            # min_norm=0.2,
            remove_duplicates=True,
        )

        ray.shutdown()
        print(f"Dumping files in {cistopic_qc_out}...")
        for sample in sorted(metadata_bc_dict.keys()):
            metadata_bc_dict[sample]["sample_id"] = sample
            metadata_bc_dict[sample].index = [
                x + "___" + sample for x in list(metadata_bc_dict[sample].index)
            ]
            with open(
                os.path.join(cistopic_qc_out, f"{sample}__metadata_bc.pkl"), "wb"
            ) as f:
                pickle.dump(metadata_bc_dict[sample], f, protocol=4)

            with open(
                os.path.join(cistopic_qc_out, f"{sample}__profile_data.pkl"), "wb"
            ) as f:
                pickle.dump(profile_data_dict[sample], f, protocol=4)
else:
    print("All samples already processed.")

2022-11-16 19:03:38,194	INFO worker.py:1509 -- Started a local Ray instance. View the dashboard at http://127.0.0.1:8265 


(compute_qc_stats_ray pid=31333) 2022-11-16 19:03:41,557 cisTopic     INFO     Reading BIO_ddseq_m1c1.FULL.fragments.tsv.gz
(compute_qc_stats_ray pid=31332) 2022-11-16 19:03:41,562 cisTopic     INFO     Reading BIO_ddseq_m1c3.FULL.fragments.tsv.gz
(compute_qc_stats_ray pid=31330) 2022-11-16 19:03:41,583 cisTopic     INFO     Reading BIO_ddseq_m1c2.FULL.fragments.tsv.gz
(compute_qc_stats_ray pid=31329) 2022-11-16 19:03:41,620 cisTopic     INFO     Reading BIO_ddseq_m1c4.FULL.fragments.tsv.gz
(compute_qc_stats_ray pid=31331) 2022-11-16 19:03:41,622 cisTopic     INFO     Reading BIO_ddseq_m1c5.FULL.fragments.tsv.gz


(compute_qc_stats_ray pid=31333) 2022-11-16 19:06:08,429 cisTopic     INFO     Computing barcode rank plot for BIO_ddseq_m1c1.FULL.fragments.tsv.gz
(compute_qc_stats_ray pid=31333) 2022-11-16 19:06:08,429 cisTopic     INFO     Counting fragments


(compute_qc_stats_ray pid=31333) 2022-11-16 19:06:25,202 cisTopic     INFO     Marking barcodes with more than 10


(compute_qc_stats_ray pid=31333) 2022-11-16 19:06:25,466 cisTopic     INFO     Returning plot data
(compute_qc_stats_ray pid=31333) 2022-11-16 19:06:25,519 cisTopic     INFO     Returning valid barcodes


...

(compute_qc_stats_ray pid=35309) 2022-11-16 20:47:16,320 cisTopic     INFO     Computing FRIP profile for VIB_hydrop_3.FULL.fragments.tsv.gz


(compute_qc_stats_ray pid=35309) 2022-11-16 20:47:17,418 cisTopic     INFO     Counting fragments


(compute_qc_stats_ray pid=35309) 2022-11-16 20:47:33,138 cisTopic     INFO     Intersecting fragments with regions


(compute_qc_stats_ray pid=35309) 2022-11-16 20:47:52,454 cisTopic     INFO     Sample VIB_hydrop_3.FULL.fragments.tsv.gz done!


Dumping files in /lustre1/project/stg_00090/scatac_benchmark/public_4_cistopic_consensus/cistopic_qc_out...


# Plot

Calculating a KDE is simultaneously expensive and scales poorly with increasing n. Therefore, I wrote a multithreaded script that divides the QC array into equal parts (interleaved to avoid biases in the order!) and performs a KDE calculation on each part. Here, Otsu thresholding is used to find the right threshold for minimum fragments and minimum TSS enrichment. ddseq samples have a significantly higher noise floor than the other samples when it comes to fragment distribution. Therefore, the otsu algorithm is performed on all barcodes with a minimum of 300 fragments for the ddseq samples, and a minimum of 100 fragments for all the other samples. I tried to perform this filtering completely independent of sample/technique (e.g. using gaussian mixture modeling, Jenks natural breaks, or multiple step Otsu thresholding) but found that no solution worked perfectly for all samples.

This is regulated by the code below in qc_plots.py:
```
min_otsu_frags_dict = {}
for fragments_file in fragments_list:
    sample = fragments_file.split("/")[-1].split(".")[0]
    tech = sample.split('_')[1]
    if tech == "ddseq":
        if sample == "BIO_ddseq_1":
            min_otsu_frags_dict[sample] = 600
        else:
            min_otsu_frags_dict[sample] = 300
    elif tech == "hydrop":
        min_otsu_frags_dict[sample] = 300
    else:
        min_otsu_frags_dict[sample] = 100
```

In [14]:
!cat ../0_resources/scripts/qc_plots_public.py

import kde
import pycisTopic
import glob
import os
import pandas as pd
import pickle
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import multiprocess as mp
from multiprocess import Pool
import pprint as pp

def histogram(array, nbins=100):
    """
    Draw histogram from distribution and identify centers.
    Parameters
    ---------
    array: `class::np.array`
            Scores distribution
    nbins: int
            Number of bins to use in the histogram
    Return
    ---------
    float
            Histogram values and bin centers.
    """
    array = array.ravel().flatten()
    hist, bin_edges = np.histogram(array, bins=nbins, range=None)
    bin_centers = (bin_edges[:-1] + bin_edges[1:]) / 2.0
    return hist, bin_centers


def threshold_otsu(array, nbins=100, min_value=100):
    """
    Apply Otsu threshold on topic-region distributions [Otsu, 1979].
    Parameters
    ---------
    array: `class::np.array`
            Array containing the region va

Since multiprocessing does not work with jupyter notebooks, run the following code in terminal:

```
mkdir plots_qc
mkdir selected_barcodes
SIF=../0_resources/cistopic_image/20220722_pycistopic.sif
singularity exec \
    --cleanenv \
    -H $PWD:/home \
    -B /lustre1,/staging,/data,${VSC_SCRATCH},${VSC_SCRATCH}/tmp:/tmp,${HOME}/.nextflow/assets/ \
    $SIF \
    python /lustre1/project/stg_00090/scatac_benchmark/0_resources/scripts/qc_plots_public.py
```

And then open the plots:

In [15]:
metadata_bc_pkl_list = sorted(glob.glob("cistopic_qc_out/*metadata_bc.pkl"))
metadata_bc_pkl_path_dict = {}
for metadata_bc_pkl_path in metadata_bc_pkl_list:
    sample = metadata_bc_pkl_path.split("/")[-1].split("__")[0]
    metadata_bc_pkl_path_dict[sample] = metadata_bc_pkl_path

for sample in metadata_bc_pkl_path_dict.keys():
    if os.path.exists(f"selected_barcodes/{sample}_bc_passing_filters_otsu.pkl"):
        print(f"{sample} bc passing filters exists, printing img and skipping")
        display(Image(f"plots_qc/{sample}_qc_otsu.png"))
    else:
        print(
            f"{sample} bc passing filters does not exist yet, generate using qc_plots.py script!"
        )

BIO_ddseq_m1c1.FULL.fragments.tsv.gz bc passing filters does not exist yet, generate using qc_plots.py script!
BIO_ddseq_m1c2.FULL.fragments.tsv.gz bc passing filters does not exist yet, generate using qc_plots.py script!
BIO_ddseq_m1c3.FULL.fragments.tsv.gz bc passing filters does not exist yet, generate using qc_plots.py script!
BIO_ddseq_m1c4.FULL.fragments.tsv.gz bc passing filters does not exist yet, generate using qc_plots.py script!
BIO_ddseq_m1c5.FULL.fragments.tsv.gz bc passing filters does not exist yet, generate using qc_plots.py script!
BIO_ddseq_m1c6.FULL.fragments.tsv.gz bc passing filters does not exist yet, generate using qc_plots.py script!
BIO_ddseq_m1c7.FULL.fragments.tsv.gz bc passing filters does not exist yet, generate using qc_plots.py script!
BIO_ddseq_m1c8.FULL.fragments.tsv.gz bc passing filters does not exist yet, generate using qc_plots.py script!
BIO_ddseq_m2c1.FULL.fragments.tsv.gz bc passing filters does not exist yet, generate using qc_plots.py script!
B

The biorad plots look a bit weird. I want to check significant overlap in barcodes between the biorad samples to check if something may have gone wrong when I merged the fastqs per sample.

In [16]:
import pandas as pd
import glob

In [17]:
df_merged = pd.DataFrame()
for file in glob.glob("selected_barcodes/BIO*.tsv"):
    df = pd.read_csv(file, sep="\t", index_col=0)
    df["raw_bc"] = [x.split("___")[0] for x in df.index]
    df["sample"] = [x.split("___")[1].replace("fragments.tsv.gz", "") for x in df.index]
    df_merged = pd.concat([df_merged, df])

In [18]:
df_merged

""


<span id="papermill-error-cell" style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">Execution using papermill encountered an exception here and stopped:</span>

In [19]:
test = df_merged.groupby("sample")["raw_bc"].value_counts()

KeyError: 'sample'

In [ ]:
test_df = test.unstack().T

In [ ]:
test_df.sum(axis=1).sort_values(ascending=False)

In [ ]:
test_df

In [ ]:
for file in glob.glob("selected_barcodes/BIO*.tsv"):
    print(file)

In [ ]:
from upsetplot import generate_counts

In [ ]:
import pandas as pd
from upsetplot import plot
set_names = ['set1', 'set2', 'set3']
all_elems = set1.union(set2).union(set3)
df = pd.DataFrame([[e in set1, e in set2, e in set3] for e in all_elems], columns = set_names)
df_up = df.groupby(set_names).size()
plot(df_up, orientation='horizontal')

In [ ]:
all_elems = list(set().union(*sets))
df = pd.DataFrame([[e in st for st in sets] for e in all_elems], columns = set_names)

In [ ]:
import numpy as np

In [ ]:
df_merged = pd.DataFrame()
bc_set_dict = {}
for file in sorted(glob.glob("selected_barcodes/BIO*.tsv")):
    sample = file.split("/")[-1].split(".")[0]
    df = pd.read_csv(file, sep="\t", index_col=0)
    df["raw_bc"] = [x.split("___")[0] for x in df.index]
    bc_set = set(np.concatenate([x.split("_") for x in df["raw_bc"]]).ravel())

    bc_set_dict[sample] = bc_set

In [ ]:
print(sorted(bc_set_dict.keys()))
set_list = [bc_set_dict[x] for x in sorted(bc_set_dict.keys())]

In [ ]:
all_elems = list(set().union(*set_list))
df = pd.DataFrame([[e in st for st in set_list] for e in all_elems], columns = sorted(bc_set_dict.keys()))

In [ ]:
df

In [ ]:
from upsetplot import plot
df_up = df.groupby(list(bc_set_dict.keys())).size()
plot(df_up, orientation='horizontal')

Ok, so nothing suspicious.

# override s3-atac

In [ ]:
import pickle
import pandas as pd

with open("cistopic_qc_out/OHS_s3atac_mouse.FULL__metadata_bc.pkl", "rb") as f:
    df = pickle.load(f)

In [ ]:
with open(
    "selected_barcodes/OHS_s3atac_mouse.FULL_bc_passing_filters_otsu.txt", "w"
) as f:
    for x in df.index:
        f.write(x + "\n")

In [ ]:
with open(
    "selected_barcodes/OHS_s3atac_mouse.FULL_bc_passing_filters_otsu.pkl", "wb"
) as f:
    pickle.dump(df.index, f)